In [11]:
import ChooseFlie
import ModifyModel
from collections import defaultdict
from datetime import datetime
import pandas as pd
import re

In [12]:
# 1、调窗口，读取资产包案件数据
path_file = ChooseFlie.choose_file()

# 2、、选择债务种类：信用卡/贷款，读取字段采集表
# path_tmp = ChooseFlie.choose_file()

# 【待删】测试
path_tmp = "../字段采集.xlsx"

# 【待完善】测试
type_credit = "信用卡"

In [13]:
# 1、原始数据文件
df_o = pd.read_excel(path_file,dtype=str)
# 2、字段收集
df_tmp = pd.read_excel(path_tmp,sheet_name='字段采集')
# 3、全量模板
df_tmp1 = pd.read_excel(path_tmp,sheet_name='全量模板')
# 4、枚举
df_map = pd.read_excel(path_tmp,sheet_name='枚举')

In [14]:
# 字段采集映射读取+处理：
# 1、接收该文件业务类型对应的列名采集、及列名模板的映射表
df_tmp = df_tmp[df_tmp['业务类型'] == type_credit]
df_tmp = df_tmp[['列名采集','列名模板']]

my_dict = dict(zip(df_tmp['列名采集'],df_tmp['列名模板']))

In [15]:
# 2、提取所需数据的表头，以及输出字段采集中不存在的表头（为采集是否需要作参考）
col_p = []
col_dfo = []
for col in df_o.columns:
    if col in my_dict.keys():
        col_dfo.append(col)
    else:
        col_p.append(col)

col_p = pd.DataFrame(col_p)
col_p.to_excel('../data/p_col.xlsx')

In [16]:
# 创建空df:全量，且列名以案件列名模板为准
col_new = df_tmp1[[type_credit]].dropna()
col_new = col_new[type_credit].to_list()
df = pd.DataFrame(columns = col_new)

In [17]:
# 提取原始数据文件中所需数据，并修改表头
df_o = (df_o[col_dfo]).rename(columns=my_dict)

# 清洗1：删除所有值都缺失的列、清除空格
df_o = df_o.dropna(axis=1,how="all")
df_o.replace('\s+','',regex=True,inplace=True)

In [18]:
# 清洗2：调整格式
type_col = ['币种','贷款五级分类','国籍','婚姻状况','性别','学历','学位','原始债务种类','证件类型']
num = ['期数','借款期限']
money = ['未偿本金','未偿利息','未偿费用','未偿官费','未偿债权总额','贷款利率']
money2 = ['原放贷金额','实际放款额','授信金额','初始信用额度']
date = ['权利转移日/交易基准日','开户日期','逾期日','放款日期','到期日']
tel_mobile = ['住宅电话','借款人手机','单位电话']

n = df_o.shape[0]
for c in df_o.columns:
    if c in df.columns:
        if c == '至权利转移日/交易基准日的逾期天数':
            df[c] = (pd.to_numeric(df_o[c])).astype('int')
        # 1、统一类别映射
        elif c in type_col:
            df_o[c] = df_o[c].fillna('未知')
            map_data = (df_map[df_map['字段名'] == c])[['字典采集','字典映射']]
            map_dict = dict(zip(map_data['字典采集'],map_data['字典映射']))
            try:
                df[c] = df_o[c].map(map_dict)
            except:
                pass
            # for i in range(n):
            #     df.loc[i,c] = ModifyModel.card_type(df_o.loc[i,c])
        # 2、数字：整型
        elif c in num:
            for i in range(n):
                df.loc[i,c] = ModifyModel.num_periods(df_o.loc[i,c])
            df[c] = df[c].astype(int)
        # 3、金额
        elif c in money:
            df[c] = (pd.to_numeric((df_o[c].fillna(0)))).astype('float')
        # elif c in money2:
        #     df[c] = (pd.to_numeric(df_o[c])).astype('float')
        # 4、日期
        elif c in date:
            for i in range(n):
                if '00:00:00' in str(df_o.loc[i,c]):
                    df_o.loc[i,c] = (df_o.loc[i,c]).replace('00:00:00','')
                df.loc[i,c] = ModifyModel.date_p(df_o.loc[i,c])
            try:
                df[c] = pd.to_datetime(df[c],format='%Y/%m/%d %H:%M:%S').dt.strftime('%Y/%m/%d')
            except:
                pass
        # elif c in tel_mobile:
        #     df_o[c] = df_o[c].str.replace('-','')
        #     for i in range(len(df_o[c])):
        #         df.loc[i,c] = ModifyModel.tel_type(df_o.loc[i,c])   
        # 5、身份证
        # elif c in tel_mobile:
        #     for i in range(n):
        #         df.loc[i,c] = ModifyModel.tel_type(df_o.loc[i,c])
    
        else:
            df[c] = df_o[c]
    else:
        continue


In [19]:
df.to_excel('../data/new_file.xlsx',index=False)